# Data Analysis Project
#### By: Alexandre Haddad-Delaveau

## Data Sources:

- [Kaggle](https://www.kaggle.com/datasets/abrambeyer/h1b-visa-petitions-20152019/data)
- [Original Data](https://www.dol.gov/agencies/eta/foreign-labor/performance)

## Initial Setup

In [ ]:
# Import Dependencies
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Data Exploration

In [ ]:
# Load dataset into dataframe
# NOTE: Must use 'latin-1' encoding due to non utf-8 characters in dataset
df = pd.read_csv("data/h1b_disclosure_data_2015_2019.csv", encoding='latin-1')

### Print out columns

In [ ]:
df.columns

#### Conclusion:
Columns like 'CASE_NUMBER' may not be useful to our analysis. Another column which will recquire more research for better understanding is 'SOC_CODE'.

### Print columns and data type

In [ ]:
df.dtypes

#### Conclusion:
Most variables return 'object' indicating a mix of values. This could mean their are N/A values. Exceptions are YEAR which is properly formatted as int64, and PREVAILING_WAGE, which is a float64.

### Check for nulls

In [ ]:
df.isna().sum()

#### Conclusion:
Very low quantity of NA's. Many columns don't have any NA's. 

### Check for duplicates & unique values

In [ ]:
# Print value counts to check for duplicates
df['CASE_NUMBER'].value_counts()

# Print number of case IDs that are duplicated
df['CASE_NUMBER'].duplicated().sum() # 17770 duplicates

# Print percentage of case IDs that are duplicated
df['CASE_NUMBER'].duplicated().sum() / len(df['CASE_NUMBER']) * 100 # 2.07%

#### Conclusion:
There are around 2% of duplicate rows. This is not a huge amount, and these rows can probably just be deleted.

### View first couple of rows of data

In [ ]:
df.head(10)

#### Conclusion:
Data is well formatted, and there doesn't seem to be any errors in the first couple of rows. The data is very clear, and is worth further exploring, even with the 2% of duplicates & couple of N/A values in each row.

## Data Cleaning

### Remove any duplicate rows

In [ ]:
df = df.drop_duplicates()

### Remove Columns

In [ ]:
# Remove unnecessary columns for analysis
df = df.drop('CASE_NUMBER', axis=1)

# Drop columns which I do not understand
df = df.drop('SOC_CODE', axis=1)

# Remove columns with repeated data
df = df.drop('WORKSITE', axis=1)

### Calculate percentage of each column that is N/A and remove N/A's or delete column

In [ ]:
# Print percentage N/A's of columns
for col in df.columns:
    column = df[col]
    print(column.isna().sum() / len(df) * 100)

# NO COLUMNS HAVE MORE THAN 10% N/A values

# Remove columns with N/A values
df = df.dropna()

# Remove any columns where salary is 0 or integer limit
df = df[df['PREVAILING_WAGE'] > 0]

### Clean Data Formats

In [ ]:
# Replace "FULL_TIME_POSITION" (which currently has "Y" or "N") with True/False
df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].map(lambda x: x == "Y")

# Convert "PREVAILING_WAGE" into int
df["PREVAILING_WAGE"] = df["PREVAILING_WAGE"].map(lambda x: int(x))

# Remove commas at end of certain WORKCITY_CITY columns
df["WORKSITE_CITY"] = df["WORKSITE_CITY"].map(lambda x: str(x).removesuffix(","))

# Cleanup WORKSITE_STATE_ABB
worksite_abb_map = {"NEWYORK": "NY", "ILLINOIS": "IL", "PENNSYLVANIA": "PA", "MICHIGAN": "MI", "OHIO": "OH", "MINNESOTA": "MN", "INDIANA": "IN", "WISCONSIN": "WI"}
df["WORKSITE_STATE_ABB"] = df["WORKSITE_STATE_ABB"].map(lambda x: worksite_abb_map[x] if x in worksite_abb_map.keys() else x)

# Cleanup JOB_TITLE by removing &NBSP; and &AMP;
df["JOB_TITLE"] = df["JOB_TITLE"].map(lambda x: x.replace("&NBSP;", " ").replace("&AMP;", "&"))

# All other columns are clean enough for our purposes

## Exploratory Data Analysis (Univariate)

### Year Distribution

In [ ]:
# Count applications by year
applications_by_year = df["YEAR"].value_counts()
applications_by_year = applications_by_year.reset_index() # Fix suggested by ChatGPT

# Rename columns
applications_by_year.columns = ["Year", "Number of Applications"]

# Create and title plot
fig = px.bar(applications_by_year, x="Year", y="Number of Applications", title="Number of Applications by Year")

# Show plot
fig.show()



### State Distribution

In [ ]:
# Count applications by state
applications_by_state = df["WORKSITE_STATE_FULL"].value_counts()
applications_by_state = applications_by_state.reset_index() # Fix suggested by ChatGPT

# Rename columns
applications_by_state.columns = ["State", "Number of Applications"]

# Create and bar plot
fig = px.bar(applications_by_state, x="State", y="Number of Applications", title="Number of Applications by State")

# Show plot
fig.show()

### Most Popular Roles

In [ ]:
# Count applications by job title
applications_by_role = df["JOB_TITLE"].value_counts()
applications_by_role = applications_by_role.reset_index() # Fix suggested by ChatGPT

# Rename columns
applications_by_role.columns = ["Job Title", "Number of Applications"]

# Get top 10 job titles and put rest into "other" category
top_10_job_titles = applications_by_role.head(10)
other_jobs = applications_by_role.iloc[10:]
other_jobs = other_jobs.sum()
other_jobs["Job Title"] = "Other"
other_jobs = pd.DataFrame(other_jobs).transpose()
applications_by_role = pd.concat([top_10_job_titles, other_jobs])

# Create and title pie chart
fig = px.pie(applications_by_role, values="Number of Applications", names="Job Title", title="Number of Applications by Job Title")

# Show plot
fig.show()



### Top Employers

In [ ]:
# Count applications by employer
applications_by_employer = df["EMPLOYER_NAME"].value_counts()
applications_by_employer = applications_by_employer.reset_index() # Fix suggested by ChatGPT

# Rename columns
applications_by_employer.columns = ["Employer", "Number of Applications"]

# Get top 10 employers and put rest into "other" category
top_10_employers = applications_by_employer.head(10)
other_employers = applications_by_employer.iloc[10:]
other_employers = other_employers.sum()
other_employers["Employer"] = "Other"
other_employers = pd.DataFrame(other_employers).transpose()
applications_by_employer = pd.concat([top_10_employers, other_employers])

# Create and title pie chart
fig = px.pie(applications_by_employer, values="Number of Applications", names="Employer", title="Number of Applications by Employer")

# Show plot
fig.show()

### H1-B Approval Rate

In [ ]:
# Count applications by status
applications_by_status = df["CASE_STATUS"].value_counts()
applications_by_status = applications_by_status.reset_index() # Fix suggested by ChatGPT

# Rename columns
applications_by_status.columns = ["Status", "Number of Applications"]

# Create and title pie chart
fig = px.pie(applications_by_status, values="Number of Applications", names="Status", title="Number of Applications by Status")

# Show plot
fig.show()

### Type of Work (Full-Time, Part-Time, etc.)

In [ ]:
# Count applications by type of position
type_of_work = df["FULL_TIME_POSITION"].value_counts()
type_of_work = type_of_work.reset_index() # Fix suggested by ChatGPT

# Rename columns
type_of_work.columns = ["Full Time", "Number of Applications"]

# Replace true/false with "Full Time" and "Part Time"
type_of_work["Full Time"] = type_of_work["Full Time"].map(lambda x: "Full Time" if x else "Part Time")

# Create and title pie chart
fig = px.pie(type_of_work, values="Number of Applications", names="Full Time", title="Number of Applications by Type of Position")

# Show plot
fig.show()

### Salary Distribution (Full Range)

In [ ]:
# Select salaries
salaries = df["PREVAILING_WAGE"]

# Create and title violin plot
fig = px.violin(salaries, title="Distribution of Salaries", labels={"value": "Salary", "variable": "Salary Distribution"}, box=True)

# Show plot
fig.show()


### Salary Distribution (0-10M range)

In [ ]:
# Select salaries below $10,000,000
somewhat_reasonable_salaries = df[df["PREVAILING_WAGE"] < 10000000]["PREVAILING_WAGE"]

# Create and title violin plot
fig = px.violin(somewhat_reasonable_salaries, title="Distribution of Salaries (below $10,000,000)", labels={"value": "Salary", "variable": "Salary Distribution"}, box=True)

# Show plot
fig.show()


### Salary Distribution (0-1M range)

In [ ]:
# Select salaries below $1,000,000
reasonable_salaries = df[df["PREVAILING_WAGE"] < 1000000]["PREVAILING_WAGE"]

# Create and title violin plot
fig = px.violin(reasonable_salaries, title="Distribution of Salaries (below $1,000,000)", labels={"value": "Salary", "variable": "Salary Distribution"}, box=True)

# Show plot
fig.show()

### Salary Distribution (0-250k range)

In [ ]:
# Select salaries below $250,000
very_reasonable_salaries = df[df["PREVAILING_WAGE"] < 250000]["PREVAILING_WAGE"]

# Create and title violin plot
fig = px.violin(very_reasonable_salaries, title="Distribution of Salaries (below $250,000)", labels={"value": "Salary", "variable": "Salary Distribution"}, box=True)

# Show plot
fig.show()

In [ ]:
df.head()

## Exploratory Data Anlysis (Bivariate / Multivariate) - Only Bivariate because of time constraints

### Top Paying Roles

In [ ]:
# Group mean wages by job title (and filter for job titles with >100 applications)
top_paying_roles = df[["JOB_TITLE", "PREVAILING_WAGE"]].groupby("JOB_TITLE").filter(lambda x: len(x) > 100)
top_paying_roles = top_paying_roles.groupby("JOB_TITLE").mean()
top_paying_roles = top_paying_roles.reset_index() # Fix suggested by ChatGPT

# Sort by salary and select top 10 job titles
top_paying_roles = top_paying_roles.sort_values("PREVAILING_WAGE", ascending=False)
top_paying_roles = top_paying_roles.head(10)

# Rename columns
top_paying_roles.columns = ["Job Title", "Salary"]


# Create and title bar chart
fig = px.bar(top_paying_roles, x="Job Title", y="Salary", title="Average Salary by Job Title (Top 10 Job Titles, by Salary, w/ >100 Applications)")

# Show plot
fig.show()

### Top Paying Companies

In [ ]:
# Group mean wages by employer (and filter for employers with >100 applications)
top_paying_companies = df[["EMPLOYER_NAME", "PREVAILING_WAGE"]].groupby("EMPLOYER_NAME").filter(lambda x: len(x) > 100)
top_paying_companies = top_paying_companies.groupby("EMPLOYER_NAME").mean()
top_paying_companies = top_paying_companies.reset_index() # Fix suggested by ChatGPT

# Sort by salary and select top 10 employers
top_paying_companies = top_paying_companies.sort_values("PREVAILING_WAGE", ascending=False)
top_paying_companies = top_paying_companies.head(10)

# Rename columns
top_paying_companies.columns = ["Employer", "Salary"]

# Create and title bar chart
fig = px.bar(top_paying_companies, x="Employer", y="Salary", title="Average Salary by Employer (Top 10 Employers, by Salary, w/ >100 Applications)")

# Show plot
fig.show()

### Top Paying States

In [ ]:
# Group mean wages by state
top_paying_states = df[["WORKSITE_STATE_FULL", "PREVAILING_WAGE"]].groupby("WORKSITE_STATE_FULL").mean()
top_paying_states = top_paying_states.reset_index() # Fix suggested by ChatGPT

# Sort by salary and select top 10 states
top_paying_states = top_paying_states.sort_values("PREVAILING_WAGE", ascending=False)
top_paying_states = top_paying_states.head(10)

# Rename columns
top_paying_states.columns = ["State", "Salary"]

# Create and title bar chart
fig = px.bar(top_paying_states, x="State", y="Salary", title="Average Salary by State (Top 10 States, by Salary)")

# Show plot
fig.show()

### Pay by Year

In [ ]:
# Group mean wages by year
pay_by_year = df[["YEAR", "PREVAILING_WAGE"]].groupby("YEAR").mean()
pay_by_year = pay_by_year.reset_index() # Fix suggested by ChatGPT

# Rename columns
pay_by_year.columns = ["Year", "Average Salary"]

# Create and title bar chart
fig = px.bar(pay_by_year, x="Year", y="Average Salary", title="Average Salary by Year")

# Show plot
fig.show()

### Most Accepted Roles

In [ ]:
# Group status by job title (and filter for job titles with >100 applications)
most_accepted_jobs = df[["JOB_TITLE", "CASE_STATUS"]].groupby("JOB_TITLE").filter(lambda x: len(x) > 100)

# For each job title, calculate the percentage of applications that were certified
most_accepted_jobs = most_accepted_jobs.groupby("JOB_TITLE").apply(lambda x: len(x[x["CASE_STATUS"] == "CERTIFIED"]) / len(x) * 100)

# Sort by percentage and select top 10 job titles
most_accepted_jobs = most_accepted_jobs.sort_values(ascending=False)
most_accepted_jobs = most_accepted_jobs.head(10)

# Rename columns
most_accepted_jobs = most_accepted_jobs.reset_index()
most_accepted_jobs.columns = ["Job Title", "Percentage Certified"]

# Create and title bar chart
fig = px.bar(most_accepted_jobs, x="Job Title", y="Percentage Certified", title="Percentage of Applications Certified by Job Title (Top 10 Job Titles, by Percentage Certified, w/ >100 Applications)")

# Show plot
fig.show()



### Most Rejected Roles

In [ ]:
# Group status by job title (and filter for job titles with >100 applications)
most_accepted_jobs = df[["JOB_TITLE", "CASE_STATUS"]].groupby("JOB_TITLE").filter(lambda x: len(x) > 100)

# For each job title, calculate the percentage of applications that were certified
most_accepted_jobs = most_accepted_jobs.groupby("JOB_TITLE").apply(lambda x: len(x[x["CASE_STATUS"] == "CERTIFIED"]) / len(x) * 100)

# Sort by percentage and select worse 10 job titles
most_accepted_jobs = most_accepted_jobs.sort_values(ascending=True)
most_accepted_jobs = most_accepted_jobs.head(10)

# Rename columns
most_accepted_jobs = most_accepted_jobs.reset_index()
most_accepted_jobs.columns = ["Job Title", "Percentage Certified"]

# Create and title bar chart
fig = px.bar(most_accepted_jobs, x="Job Title", y="Percentage Certified", title="Percentage of Applications Certified by Job Title (Worst 10 Job Titles, by Percentage Certified, w/ >100 Applications)")

# Show plot
fig.show()

### Most Accepted Employers

In [ ]:
# Group status by employer (and filter for employers with >100 applications)
most_accepted_employers = df[["EMPLOYER_NAME", "CASE_STATUS"]].groupby("EMPLOYER_NAME").filter(lambda x: len(x) > 100)

# For each employer, calculate the percentage of applications that were certified
most_accepted_employers = most_accepted_employers.groupby("EMPLOYER_NAME").apply(lambda x: len(x[x["CASE_STATUS"] == "CERTIFIED"]) / len(x) * 100)

# Sort by percentage and select top 10 employers
most_accepted_employers = most_accepted_employers.sort_values(ascending=False)
most_accepted_employers = most_accepted_employers.head(10)

# Rename columns
most_accepted_employers = most_accepted_employers.reset_index()
most_accepted_employers.columns = ["Employer", "Percentage Certified"]

# Create and title bar chart
fig = px.bar(most_accepted_employers, x="Employer", y="Percentage Certified", title="Percentage of Applications Certified by Employer (Top 10 Employers, by Percentage Certified, w/ >100 Applications)")

# Show plot
fig.show()

### Most Rejected Employers

In [ ]:
# Group status by employer (and filter for employers with >100 applications)
most_accepted_employers = df[["EMPLOYER_NAME", "CASE_STATUS"]].groupby("EMPLOYER_NAME").filter(lambda x: len(x) > 100)

# For each employer, calculate the percentage of applications that were certified
most_accepted_employers = most_accepted_employers.groupby("EMPLOYER_NAME").apply(lambda x: len(x[x["CASE_STATUS"] == "CERTIFIED"]) / len(x) * 100)

# Sort by percentage and select worse 10 employers
most_accepted_employers = most_accepted_employers.sort_values(ascending=True)
most_accepted_employers = most_accepted_employers.head(10)

# Rename columns
most_accepted_employers = most_accepted_employers.reset_index()
most_accepted_employers.columns = ["Employer", "Percentage Certified"]

# Create and title bar chart
fig = px.bar(most_accepted_employers, x="Employer", y="Percentage Certified", title="Percentage of Applications Certified by Employer (Worst 10 Employers, by Percentage Certified, w/ >100 Applications)")

# Show plot
fig.show()

### Rejection for Full-Time vs Part-Time

In [ ]:
# Group status by job type (Full Time/Part Time)
job_type_rate = df[["FULL_TIME_POSITION", "CASE_STATUS"]].groupby("FULL_TIME_POSITION")

# For each job type, calculate the percentage of applications that were certified
job_type_rate = job_type_rate.apply(lambda x: len(x[x["CASE_STATUS"] == "CERTIFIED"]) / len(x) * 100)

# Rename columns
job_type_rate = job_type_rate.reset_index()
job_type_rate.columns = ["Full Time", "Percentage Certified"]

# Replace true/false with "Full Time" and "Part Time"
job_type_rate["Full Time"] = job_type_rate["Full Time"].map(lambda x: "Full Time" if x else "Part Time")

# Create and title bar chart
fig = px.bar(job_type_rate, x="Full Time", y="Percentage Certified", title="Percentage of Applications Certified by Job Type")

# Show plot
fig.show()


## Statistical Analysis

### How has the mean salary changed over time?
Salary dipped slightly from 2015-2016, but then steadily increased from 2016-2018. In 2019 it dipped slightly again to levels below that of 2018. The salary average salary in 2015 was $88k. In 2018 it was $110k, and in 2019 it was $95k.

### How does a part-time/full-time job affect acceptance rate?
Whether a job is full-time or part-time doesn't seem to significantly impact the acceptance rate. The acceptance rate for full-time jobs is 89.5%, and for part-time jobs it is 88.5%. We can conclude that the type of job doesn't significantly impact the acceptance rate.

### How does the mean salary change by state?
The mean salary varies significantly by state. The state with the highest mean salary is Minnesota, with a mean salary of $106k. The state with the lowest mean salary is Wisconsin, with a mean salary of $74k.

In [ ]:
df.head(1).transpose().to_markdown("data.md")